# Jugando juegos de Atari usando DQN 
Atari 2600 es una consola de videojuegos popular de una compañia de juegos llamada Atari. Esta consola poseia muchos titulos populares como pong, space invaders, Ms Pacman, entre otros. En este ejercicio en particular se construye una red Q profunda (DQN por sus siglas en inglés) para jugar Ms Pacman.

## Arquitectura de DQN
En el ambiente de Atari, la imagen del juego en pantalla es el estado del ambiente. Por tanto, la imágen del juego es la que se introduce como entrada a la DQN y retorna el valor Q de todas las acciones en el estado. Ya que se están ocupando imágenes, en ves de unar una red neuronal profunda simple para aproximar el valor de Q, se ocupa una red convolucional ya que estas son muy efectivas para tratamiento de imágenes.

Por tanto, la DQN es una red neuronal convolucional. Se alimenta la imágen del juego como una entrada en la red neuronal convolucional y da como salida el valor Q de todas las acciones en el estado.

Como se muestra en la siguiente figura, las capas convolucionales extraen las caracteristicas de la imagen y producen un mapa de caracteristicas. Despues, se aplana este mapa de caracteristicas y se alimenta la red con este mapa. La parte de feedforward de la red toma este mapa aplanado como una entrada y retorna el valor Q de todas las acciones en el estado.


![title](Images/4.png)

Algo a considerar es que no hay operaciones de pooling. La operacion de pooling es util cuando se realizan tareas tales como deteccion de objetos, clasificacion de imágenes y otras cosas donde no se considera la posición del objeto en la imágen y solo se desea saber si el objeto deseado está presente en esta. 

Como lo que se desea es saber el estado del juego, la posición de los objetos es muy importante, por esto no se realiza una operación de pooling.


## Implementando la red DQN

Primero se importan las librerias necesarias. El autor realizó la prueba con tensorflow 2.0, en mi caso lo hice con la version 2.3.1 y usando la gpu de mi computadora.

In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
print(tf.__version__)

2.3.1


In [2]:
import random
import gym
import numpy as np
from collections import deque
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

Ahora, se crea el ambiente de Ms Pacman usando gym:

In [3]:
env = gym.make("MsPacman-v0")

Se establece el tamaño del estado:

In [4]:
state_size = (88, 80, 1)

Se obtiene el numero de acciones posibles por el agente

In [5]:
action_size = env.action_space.n

## Preprocesamiento de la imagen del juego

Ya se mencionó que se alimentaria la red DQN con el estado del juego (imagen de la pantalla del juego), sin embarbo, alimentar la red directamente con la imagen cruda no es eficiente, ya que el tamaño de esta es de 210x160x3 lo cual es computacionalmente muy costoso.
Para evitar esto, se hace un preprocesamiento a la imagen y despues se introduce a la red DQN. Primero, se corta y redimensiona el tamaño de la imágen, convirtiendo la imágen a escala de grises, normalizando y redimensionando la imágen a 88x80x1. 

Para esto se ocupa la siguiente funcion llamada preprocess_state

In [6]:
color = np.array([210, 164, 74]).mean()

def preprocess_state(state):

    #crop and resize the image
    image = state[1:176:2, ::2]

    #convert the image to greyscale
    image = image.mean(axis=2)

    #improve image contrast
    image[image==color] = 0

    #normalize the image
    image = (image - 128) / 128 - 1
    
    #reshape the image
    image = np.expand_dims(image.reshape(88, 80, 1), axis=0)

    return image

## Construyendo la DQN 

Ahora, se construye la red Q profunda. Esta red recibe una imágen de entrada, la cual procesa con capas convolucionales y da como resultado los valores Q.

Se definen tres capas convolucionales. Estas capas convolucionales extraen las caracteristicas de la imágen, mas adelante se aplanan los datos obtenidos y se introducen a dos capas densas, y la ultima capa retorna los valores Q.


In [7]:
class DQN:
    def __init__(self, state_size, action_size):
        
        #define el tamaño del estado
        self.state_size = state_size
        
        #define el tamaño de la accion
        self.action_size = action_size
        
        #define el buffer de rejuego
        self.replay_buffer = deque(maxlen=5000)
        
        #define el factor de descuento
        self.gamma = 0.9  
        
        #define el valor epsilon
        self.epsilon = 0.8   
        
        #define el ratio de actualizacion con el que se desea actualizar el objetivo de la red
        self.update_rate = 1000    
        
        #define la red principal
        self.main_network = self.build_network()
        
        #define el objetivo de la red
        self.target_network = self.build_network()
        
        #copia los pesos de la red principal a la red de objetivo
        self.target_network.set_weights(self.main_network.get_weights())
        

    #Ahora se define una funcion llamada build_network la cual es necesaria para la DQN

    def build_network(self):
        model = Sequential()
        model.add(Conv2D(32, (8, 8), strides=4, padding='same', input_shape=self.state_size))
        model.add(Activation('relu'))
        
        model.add(Conv2D(64, (4, 4), strides=2, padding='same'))
        model.add(Activation('relu'))
        
        model.add(Conv2D(64, (3, 3), strides=1, padding='same'))
        model.add(Activation('relu'))
        model.add(Flatten())


        model.add(Dense(512, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        
        model.compile(loss='mse', optimizer=Adam())

        return model


    #La red se entrena mediante pequeños bloques de muestras de trancisiones del buffer de rejuego.
    #Por tanto, se define una función llamada store_transition la cual guarda la
    # informacion de las trancisiones en el buffer de rejuego

    def store_transistion(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))
        

    #Para tener un control del balance exploración-explotacion, se seleccionan las acciones
    #usando la política epsilon-greedy. Por tanto, se define esta funcion para seleccionar
    #una accion usando la politica epsilon-greedy
    
    def epsilon_greedy(self, state):
        if random.uniform(0,1) < self.epsilon:
            return np.random.randint(self.action_size)
        
        Q_values = self.main_network.predict(state)
        
        return np.argmax(Q_values[0])

    
    #Entrenamiento de la red
    def train(self, batch_size):
        
        #muestrea un minibloque de transiciones desde el buffer de replayr
        minibatch = random.sample(self.replay_buffer, batch_size)
        
        #Calcula el valor de Q usando la red de objetivo
        for state, action, reward, next_state, done in minibatch:
            if not done:
                target_Q = (reward + self.gamma * np.amax(self.target_network.predict(next_state)))
            else:
                target_Q = reward
                
            #Calcula el valor de Q usando la red principal
            Q_values = self.main_network.predict(state)
            
            Q_values[0][action] = target_Q
            
            #Entrena la red principal
            self.main_network.fit(state, Q_values, epochs=1, verbose=0)
            
    #actualiza los pesos de la red de objetivo copiandolos desde la red principal
    def update_target_network(self):
        self.target_network.set_weights(self.main_network.get_weights())

## Entrenando la red

Ahora, se entrena la red, para esto, primero se establece el numero de episodios deseados.
en el codigo original realizan más de 10 episodios, si mal no recuerdo eran 50 o 100, pero son muy tardados por lo que yo solo ejecuté 10 episodios.

In [8]:
num_episodes = 10

Se define el numero de pasos de tiempo

In [9]:
num_timesteps = 20000

Se define el tamaño del bloque

In [10]:
batch_size = 8

Se establece el numero de imágenes de la pantalla despues del juego que se desean considerar

In [11]:
num_screens = 4     

Se instancia la clase DQN

In [12]:
dqn = DQN(state_size, action_size)

In [13]:
done = False
time_step = 0

#for para cada episodio
for i in range(num_episodes):
    
    #establece el valor de retorno en 0
    Return = 0
    
    #realiza el preprocesamiento de la imagen en pantalla, además de reiniciar el entorno
    state = preprocess_state(env.reset())

    #for que recorre cada paso de tiempo del episodio
    for t in range(num_timesteps):
        
        #renderiza el entorno
        env.render()
        
        #actualiza el paso de tiempo
        time_step += 1
        
        #actualiza la red de objetivo
        if time_step % dqn.update_rate == 0:
            dqn.update_target_network()
        
        #seleciona la accion
        action = dqn.epsilon_greedy(state)
        
        #realiza la accion seleccionada
        next_state, reward, done, _ = env.step(action)
        
        #preprocesa el siguiente estado
        next_state = preprocess_state(next_state)
        
        #guarda la información de transición
        dqn.store_transistion(state, action, reward, next_state, done)
        
        #actualiza el estado actual a el estado siguiente
        state = next_state
        
        #actualiza el valor de retorno añadiendo el valor de la recompenzza
        Return += reward
        
        #si el episodio ha terminado, muestra el valor de retorno y sale del for interno
        if done:
            print('Episode: ',i, ',' 'Return', Return)
            break
            
        #si el numero de transiciones del buffer de rejuego es mayor
        #que el tamaño del bloque, entonces se entrena la red
        if len(dqn.replay_buffer) > batch_size:
            dqn.train(batch_size)


Episode:  0 ,Return 290.0
Episode:  1 ,Return 140.0
Episode:  2 ,Return 440.0
Episode:  3 ,Return 270.0
Episode:  4 ,Return 200.0
Episode:  5 ,Return 280.0
Episode:  6 ,Return 450.0
Episode:  7 ,Return 330.0
Episode:  8 ,Return 300.0
Episode:  9 ,Return 210.0


# Conclusiones

Antes de correr este código, ya habia probado otro ejemplo usando DQN tambien utilizando la libreria de gym, en ese caso fue un péndulo invertido y el numero de acciones y valores del ambiente es menor por lo que era mas sencillo de comprender. Este ejemplo por otra parte es mas enriquecedor ya que trabaja a partir de una imagen de entrada, lo cual se acerca más a un ambiente real en el que un agente mediante vision artificial pueda observar su ambiente y realizar acciones
Obviamente a mayor complejidad con la entrada, el tiempo de entrenamiento tambien aumenta, estos 10 episodios le llevó aproximadamente una hora a mi computadora usando la GPU, y lo recomendado era hacer más de 50 episodios para comenzar a tener un agente "inteligente". Aunque no se necesita una base de datos para el entrenamiento, el aprendizaje aun lleva mucho tiempo por las "simulaciones" que realiza el agente.